In [1]:
import spacy
import os
import numpy as np
import pandas as pd
from google.colab import drive
import requests

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/CAPSTONE/data')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/CAPSTONE/data'

In [3]:
os.listdir()

['Will_1k_set_1.csv',
 'Will_1k_set_10.csv',
 'Will_1k_set_11.csv',
 'Will_1k_set_12.csv',
 'Will_1k_set_13.csv',
 'Will_1k_set_14.csv',
 'Will_1k_set_15.csv',
 'Will_1k_set_16.csv',
 'Will_1k_set_17.csv',
 'Will_1k_set_18.csv',
 'Will_1k_set_19.csv',
 'Will_1k_set_2.csv',
 'Will_1k_set_20.csv',
 'Will_1k_set_21.csv',
 'Will_1k_set_22.csv',
 'Will_1k_set_23.csv',
 'Will_1k_set_24.csv',
 'Will_1k_set_25.csv',
 'Will_1k_set_3.csv',
 'Will_1k_set_4.csv',
 'Will_1k_set_9.csv',
 'Will_1k_set_5.csv',
 'Will_1k_set_6.csv',
 'Will_1k_set_7.csv',
 'Will_1k_set_8.csv',
 'combined_data.csv',
 'bert_tag_prediction_model.pth',
 'optimizer_state.pth',
 'data_final.csv',
 'results',
 'logs']

In [39]:
# import csv file, drop irrelevant column
data = pd.read_csv('Will_1k_set_1.csv')
#data = pd.read_csv('combined_data.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,file_name,wiki_commons_id,wiki_data_id,depicts,description,parsed caption,commons_description
0,File:Plac Wielkopolski w Poznaniu - czerwiec 2...,60332930,['Q11819181'],['Wielkopolski Square in Poznań'],"['city square in Poznań, Poland']",Plac Wielkopolski w Poznaniu.\n\n,"{'DateTime': {'value': '2017-06-24 03:53:38', ..."
1,File:2018-10-10 Gymnastics at 2018 Summer Yout...,77382844,['Q93888038'],['Yin Dehang'],['Chinese artistic gymnast'],Horizontal bar qualification of the boys' arti...,"{'DateTime': {'value': '2019-03-16 21:07:55', ..."
2,File:CP 00118destructions rue de Tournai 1914.jpg,84142136,['Q39074508'],['Siege of Lille'],['1914 siege'],Lille (France - Nord department) - Destruction...,"{'DateTime': {'value': '2019-11-19 16:24:13', ..."
3,"File:Waycross, Georgia Historic District (40).jpg",86478066,['Q30622481'],['Waycross Historic District'],"['historic district in Waycross, Georgia']","Waycross Historic District, Waycross, Georgia\...","{'DateTime': {'value': '2020-02-01 01:11:58', ..."
4,File:J31 867 Kraftwerkskaverne.jpg,138279755,['Q963375'],['diesel generator'],['combination of a diesel engine with an elect...,Zwei Dieselgeneratoren für die Gesamtversorgun...,"{'DateTime': {'value': '2023-07-16 20:54:40', ..."


In [40]:
data.iloc[1,3] = "['Yin Dehang', '2018 Summer Youth Olympics']"

In [41]:
data.shape[0]

1000

In [7]:
# download large SpaCy Model for testing purposes
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:

'''
def clean_and_lemmatize(phrase, doc):
    articles = ['the', 'a', 'an']
    words = phrase.split()
    if words and words[0].lower() in articles:
        words = words[1:]
    lemmatized_words = []
    for word in words:
        tokens = [token for token in doc if token.text.lower() == word.lower()]
        if tokens:
            lemmatized_words.append(tokens[0].lemma_)  # Append lemma to the list
        else:
            lemmatized_words.append(word)  # Append original word if no token found
    return ' '.join(lemmatized_words)
  '''
import spacy
import re

nlp = spacy.load("en_core_web_lg")
def clean_and_lemmatize(phrase, doc):
    articles = ['the', 'a', 'an']
    words = phrase.split()
    # Remove the first word if it is an article
    if words[0].lower() in articles:
        words = words[1:]
    # Lemmatize each word in the phrase
    lemmatized_words = []
    for word in words:
        # Find the token for this word (use the original text for matching)
        tokens = [token for token in doc if token.text.lower() == word.lower()]
        if tokens:
            # Use the lemma of the first matching token
            lemma = tokens[0].lemma_
        else:
            lemma = word  # Fallback to the original word if not found
        lemmatized_words.append(lemma)
    return ' '.join(lemmatized_words)

def process_text(text):
  try:
    #stripped_text = text.strip("['").strip("']")
    #doc = nlp(stripped_text)
    doc = nlp(text)
    url_pattern = r'https?://[^\S\n]*(?:[^\s<>"]|[^\S\n])+'
    text = re.sub(url_pattern, '', text, flags=re.MULTILINE)
    #noun_chunks = [clean_and_lemmatize(chunk.text, doc) for chunk in doc.noun_chunks]
    noun_chunks = [clean_and_lemmatize(chunk.text, doc) for chunk in doc.noun_chunks]
    named_entities = [clean_and_lemmatize(entity.text, doc) for entity in doc.ents]
    tags = list(set(noun_chunks + named_entities))
    #filtered_tags = [tag for tag in tags if not (tag.lower().startswith('http') or not tag[0].isalnum() or tag.isdigit())]
    filtered_tags = [tag for tag in tags if not (tag.lower().startswith('http') or not tag.isalnum() or tag.isdigit())]
    return filtered_tags
  except:
    return 'no tags found'

# Apply the function to the DataFrame
#results = data['parsed caption'].apply(process_text)
#print(results)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [43]:
#data['extracted_tags'] = data['parsed caption'].apply(process_text)
data['extracted_tags'] = data['parsed caption'].apply(process_text)

In [99]:
string = new_df.iloc[1,5]

In [105]:
string

"Horizontal bar qualification of the boys' artistic gymnastics at the 2018\nSummer Youth Olympics in Buenos Aires on 10 October 2018. Depicted: Yin\nDehang.\n\n"

In [103]:
process_text(string)

[]

In [10]:
data.head()

,file_name,wiki_commons_id,wiki_data_id,depicts,description,parsed caption,commons_description,extracted_tags
0,File:Plac Wielkopolski w Poznaniu - czerwiec 2...,60332930,['Q11819181'],['Wielkopolski Square in Poznań'],"['city square in Poznań, Poland']",Plac Wielkopolski w Poznaniu.\n\n,"{'DateTime': {'value': '2017-06-24 03:53:38', ...",[Plac Wielkopolski w Poznaniu]
1,File:2018-10-10 Gymnastics at 2018 Summer Yout...,77382844,['Q93888038'],"['Yin Dehang', '2018 Summer Youth Olympics']",['Chinese artistic gymnast'],Horizontal bar qualification of the boys' arti...,"{'DateTime': {'value': '2019-03-16 21:07:55', ...","[Buenos Aires, 10 October, Yin Dehang, 2018 Su..."
2,File:CP 00118destructions rue de Tournai 1914.jpg,84142136,['Q39074508'],['Siege of Lille'],['1914 siege'],Lille (France - Nord department) - Destruction...,"{'DateTime': {'value': '2019-11-19 16:24:13', ...","[under-german, October, the-department, histor..."
3,"File:Waycross, Georgia Historic District (40).jpg",86478066,['Q30622481'],['Waycross Historic District'],"['historic district in Waycross, Georgia']","Waycross Historic District, Waycross, Georgia\...","{'DateTime': {'value': '2020-02-01 01:11:58', ...","[America](https://en.wikipedia.org, place, Way..."
4,File:J31 867 Kraftwerkskaverne.jpg,138279755,['Q963375'],['diesel generator'],['combination of a diesel engine with an elect...,Zwei Dieselgeneratoren für die Gesamtversorgun...,"{'DateTime': {'value': '2023-07-16 20:54:40', ...","[Maschinen mit Druckgas, der Wand, Zwei Diesel..."


In [44]:
data = data[data['extracted_tags'] != 'no tags found']

In [45]:
new_df = data.copy()
data = data[['depicts', 'extracted_tags']]

In [46]:
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available.")

GPU available: Tesla T4


In [47]:
datalist = []
for _, row in data.iterrows():
    # Join the list of depicts into a single string, separating by a space or a period
    depicts = ' '.join(row['depicts'])
    for tag in row['extracted_tags']:
        # Check if the tag is a substring of the 'depicts'
        label = 1 if tag in depicts else 0
        datalist.append({'depicts': depicts, 'tag': tag, 'label': label})

expanded_df = pd.DataFrame(datalist)

In [54]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.20, random_state=42)

print("Training Data:")
print(len(train_df))
print("\nTesting Data:")
print(len(test_df))

Training Data:
770

Testing Data:
193


In [49]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [59]:
def evaluate_model(dataframe):
    results_list = []
    for _, row in dataframe.iterrows():
        depicts = row['depicts']
        tags = row['extracted_tags']
        # Ensure both 'depicts' and 'tags' are not empty
        if depicts and tags:
            # Predicting probabilities
            prediction = classifier(
                depicts,
                candidate_labels=tags,
                hypothesis_template="This image is described as {}.",
                multi_label=True
            )
            # Creating the output dictionary for this observation
            tag_probabilities = {tag: prob for tag, prob in zip(prediction['labels'], prediction['scores'])}
            results_list.append(tag_probabilities)
        else:
            # Handle the case where 'depicts' or 'tags' are empty
            results_list.append({"error": "Empty depicts or tags"})

    return results_list

In [60]:
train_results = evaluate_model(train_df)
print(train_results)

[{'error': 'Empty depicts or tags'}, {'ADN': 0.020425252616405487, 'Ostrokrzew': 0.010315170511603355, 'inventory': 0.005940333474427462}, {'Napoleon': 0.9991390705108643, 'part': 0.9841270446777344, 'source': 0.8122299313545227, 'series': 0.36406445503234863, 'companion': 0.0573003925383091, 'ADN': 0.05438261106610298, 'field': 0.014127996750175953, 'inventory': 0.006425544619560242, 'Waterloo': 0.0005987246986478567, 'Wellington': 0.0003077960864175111}, {'Trentino': 0.03211657330393791, 'Italy': 0.013469738885760307, 'Agrone': 0.013321289792656898}, {'this': 0.8565976619720459, 'image': 0.18427234888076782}, {'Deutscher': 0.5556896924972534}, {'Carolina': 0.9460090398788452, 'localizado': 0.7191158533096313, 'MA': 0.07883813977241516, 'ADN': 0.016887327656149864, 'inventory': 0.01386864110827446, 'Estreito': 0.0009707824210636318, 'Riachão': 0.0008572899387218058}, {'error': 'Empty depicts or tags'}, {'Copse': 0.6744462251663208, 'this': 0.2719849944114685, 'commons': 0.148101791739

In [55]:
test_df.iloc[0,]

depicts           ['Jana Lisa Rother', 'Frithjof Seidel']
extracted_tags                        , I, Abgebildet]
Name: 552, dtype: object

In [35]:
'charing_cross_station_-'[0].isalnum()

True

In [62]:
train_results[1]

{'ADN': 0.020425252616405487,
 'Ostrokrzew': 0.010315170511603355,
 'inventory': 0.005940333474427462}

In [ ]:
train_results

In [64]:
######

In [65]:
#####


In [66]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_tag_as_depicts(tag):
    # Treat the tag as both 'depicts' and 'tag'
    texts = tag + " [SEP] " + tag  # You might choose to handle this concatenation differently based on model training
    encoded_dict = tokenizer(texts, max_length=128, padding='max_length', truncation=True, return_tensors='tf')
    return encoded_dict['input_ids'], encoded_dict['attention_mask']

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [67]:
def predict_tag_relevance_single(model, tag):
    input_ids, attention_mask = encode_tag_as_depicts(tag)
    prediction = model.predict([input_ids, attention_mask])
    return prediction[0][0]

In [70]:
tags = ['Buenos Aires', '10 October', 'Yin Dehang', '2018 Summer Youth Olymnpics']
predictions = {tag: predict_tag_relevance_single(classifier, tag) for tag in tags}

for tag, prob in predictions.items():
    print(f"The probability that '{tag}' is a relevant tag on its own is {prob:.2f}")

TypeError: object of type 'NoneType' has no len()

In [71]:
def predict_tag_relevance_single(classifier, tag, context="This is a description about"):
    """ Predict the relevance of a single tag given a context.

    Args:
        classifier (pipeline): The zero-shot classification pipeline.
        tag (str): The tag to evaluate.
        context (str): A context string to use for prediction.

    Returns:
        float: The relevance probability of the tag.
    """
    try:
        # Ensuring the tag is used as a label and context is a non-empty sequence
        if tag and context:
            prediction = classifier(
                context,  # This is the sequence where the tag's relevance is checked
                candidate_labels=[tag],  # Single tag as a list of one label
                hypothesis_template="This text is about {}.",  # Simple hypothesis template
                multi_label=False  # Assuming non-multi label as we're checking one tag at a time
            )
            return prediction['scores'][0]  # Return the relevance probability of the tag
        else:
            return 0.0  # Return zero probability if the tag or context is empty
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0.0

# Example usage
tags = ['Buenos Aires', '10 October', 'Yin Dehang', '2018 Summer Youth Olympics']
predictions = {tag: predict_tag_relevance_single(classifier, tag) for tag in tags}

#for tag, prob in predictions.items():
#    print(f"The probability that '{tag}' is a relevant tag on its own is {prob:.2f}")


The probability that 'Buenos Aires' is a relevant tag on its own is 0.61
The probability that '10 October' is a relevant tag on its own is 0.16
The probability that 'Yin Dehang' is a relevant tag on its own is 0.44
The probability that '2018 Summer Youth Olympics' is a relevant tag on its own is 0.18


In [72]:
predictions

{'Buenos Aires': 0.6138836145401001,
 '10 October': 0.16300183534622192,
 'Yin Dehang': 0.43886271119117737,
 '2018 Summer Youth Olympics': 0.182575985789299}

In [91]:
import requests

def get_qnum(tag):
    try:
        url = f'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={tag}&languages=en&props=labels&format=json'
        response = requests.get(url).json()
        qnum = next(iter(response['entities']))
        if qnum.startswith('Q'):
            return qnum
        else:
            return None
    except:
        return None

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        qnum = get_qnum(key)
        if qnum:
            output_dict[qnum] = value
    return output_dict

In [92]:
def filter_tags_by_threshold(predictions_dict, threshold):
    """
    Filters a dictionary of tag probabilities, returning only those entries where
    the probability is at least as large as the specified threshold.

    Args:
        predictions_dict (dict): Dictionary with tags as keys and probabilities as values.
        threshold (float): The threshold probability; only tags with probabilities
                           equal to or greater than this value are returned.

    Returns:
        dict: A filtered dictionary of tags that meet or exceed the threshold.
    """
    # Create a new dictionary with only the tags that meet or exceed the threshold
    filtered_dict = {tag: prob for tag, prob in predictions_dict.items() if prob >= threshold}
    return filtered_dict

In [93]:
def final(test, threshold=0.0):
  predictions = {tag: predict_tag_relevance_single(classifier, tag) for tag in tags}
  qnum_dict = process_dict(predictions)
  filtered_dict = filter_tags_by_threshold(qnum_dict, threshold)
  return filtered_dict

In [108]:
test = ['Buenos Aires', '10 October', 'Yin Dehang', '2018 Summer Youth Olympics']

final(test)#, threshold = 0.2)

{'Q1486': 0.6138836145401001,
 'Q93888038': 0.43886271119117737,
 'Q870879': 0.182575985789299}

In [106]:
string

"Horizontal bar qualification of the boys' artistic gymnastics at the 2018\nSummer Youth Olympics in Buenos Aires on 10 October 2018. Depicted: Yin\nDehang.\n\n"

In [107]:
string.process_text()

AttributeError: 'str' object has no attribute 'process_text'